# Contradictory, My Dear Watson

### Detecting contradiction and entailment in multilingual text using TPUs

**Kaggle link:** https://www.kaggle.com/competitions/contradictory-my-dear-watson/overview

### Description

"…when you have eliminated the impossible, whatever remains, however improbable, must be the truth"
-Sir Arthur Conan Doyle

Our brains process the meaning of a sentence like this rather quickly.

We're able to surmise:

Some things to be true: "You can find the right answer through the process of elimination.”
Others that may have truth: "Ideas that are improbable are not impossible!"
And some claims are clearly contradictory: "Things that you have ruled out as impossible are where the truth lies."
Natural language processing (NLP) has grown increasingly elaborate over the past few years. Machine learning models tackle question answering, text extraction, sentence generation, and many other complex tasks. But, can machines determine the relationships between sentences, or is that still left to humans? If NLP can be applied between sentences, this could have profound implications for fact-checking, identifying fake news, analyzing text, and much more.

### The Challenge:
If you have two sentences, there are three ways they could be related: one could entail the other, one could contradict the other, or they could be unrelated. Natural Language Inferencing (NLI) is a popular NLP problem that involves determining how pairs of sentences (consisting of a premise and a hypothesis) are related.

Your task is to create an NLI model that assigns labels of 0, 1, or 2 (corresponding to entailment, neutral, and contradiction) to pairs of premises and hypotheses. To make things more interesting, the train and test set include text in fifteen different languages! You can find more details on the dataset by reviewing the Data page.

Today, the most common approaches to NLI problems include using embeddings and transformers like BERT. In this competition, we’re providing a starter notebook to try your hand at this problem using the power of Tensor Processing Units (TPUs). TPUs are powerful hardware accelerators specialized in deep learning tasks, including Natural Language Processing. Kaggle provides all users TPU Quota at no cost, which you can use to explore this competition. Check out our TPU documentation and Kaggle’s YouTube playlist for more information and resources.

### Recommended Tutorial
We highly recommend this excellent tutorial on using KerasNLP to solve this problem, from the Keras team as well as Ana Sofia Uzsoy’s Tutorial that walks you through creating your very first submission step by step with TPUs and BERT.
    This is a great opportunity to flex your NLP muscles and solve an exciting problem!

### Disclaimer: 
The dataset for this competition contains text that may be considered profane, vulgar, or offensive.

### Evaluation

#### Goal
Your goal is to predict whether a given hypothesis is related to its premise by contradiction, entailment, or whether neither of those is true (neutral).
For each sample in the test set, you must predict a 0, 1, or 2 value for the variable.

Those values map to the logical condition as:

0 == entailment
1 == neutral
2 == contradiction

#### Metric
Your score is the percentage of relationships you correctly predict. This is known as accuracy.


#### Submission File Format
You should submit a csv file with exactly 5195 entries plus a header row. Your submission will show an error if you have extra columns (beyond id and prediction) or rows.

The file should have exactly 2 columns:

id (sorted in any order)
prediction (contains your predictions: 0 for entailment, 1 for neutral, 2 for contradiction)

### id,prediction

c6d58c3f69,1
cefcc82292,1
e98005252c,1
58518c10ba,1
c32b0d16df,1
Etc.

You can download an example submission file (sample_submission.csv) on the Data page.

### Code Submission Requirement

In this code competition, your submission.csv file must be generated as an output from a Kaggle notebook. For details on how to submit from a notebook, review the FAQ on "How do I make a submission?"

### Dataset Description
In this Getting Started Competition, we’re classifying pairs of sentences (consisting of a premise and a hypothesis) into three categories - entailment, contradiction, or neutral. Let’s take a look at an example of each of these cases for the following premise:

He came, he opened the door and I remember looking back and seeing the expression on his face, and I could tell that he was disappointed.

#### Hypothesis 1:

Just by the look on his face when he came through the door I just knew that he was let down.

We know that this is true based on the information in the premise. So, this pair is related by entailment.


#### Hypothesis 2:

He was trying not to make us feel guilty but we knew we had caused him trouble.

This very well might be true, but we can’t conclude this based on the information in the premise. So, this relationship is neutral.

#### Hypothesis 3:

He was so excited and bursting with joy that he practically knocked the door off it's frame.

We know this isn’t true, because it is the complete opposite of what the premise says. So, this pair is related by contradiction.

This dataset contains premise-hypothesis pairs in fifteen different languages, including:
Arabic, Bulgarian, Chinese, German, Greek, English, Spanish, French, Hindi, Russian, Swahili, Thai, Turkish, Urdu, and Vietnamese.

#### Files:
train.csv: This file contains the ID, premise, hypothesis, and label, as well as the language of the text and its two-letter abbreviation

test.csv: This file contains the ID, premise, hypothesis, language, and language abbreviation, without labels.

sample_submission.csv: This is a sample submission file in the correct format:
id: a unique identifier for each sample
    label: the classification of the relationship between the premise and hypothesis (0 for entailment, 1 for neutral, 2 for contradiction)

Special thanks to Tensorflow Datasets (TFDS) for providing this and many other useful datasets! For more information, visit: https://www.tensorflow.org/datasets

Disclaimer: The dataset for this competition contains text that may be considered profane, vulgar, or offensive.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import *
from sklearn.linear_model import *
from math import *
from nltk import *
import warnings
warnings.filterwarnings("ignore")
from deep_translator import *
import os
import nltk

In [2]:
nltk.download("all")

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /Users/amith/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     /Users/amith/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /Users/amith/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /Users/amith/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /Users/amith/nltk_data...
[nltk_data]    |   Package basque_grammars is already up-to-date!
[nltk_data]    | Downloading package bcp47 to
[nltk_data]    |

True

In [3]:
df_train = pd.read_csv("data/train.csv")
df_train.head()

,id,premise,hypothesis,lang_abv,language,label
0,5130fd2cb5,and these comments were considered in formulat...,The rules developed in the interim were put to...,en,English,0
1,5b72532a0b,These are issues that we wrestle with in pract...,Practice groups are not permitted to work on t...,en,English,2
2,3931fbe82a,Des petites choses comme celles-là font une di...,J'essayais d'accomplir quelque chose.,fr,French,0
3,5622f0c60b,you know they can't really defend themselves l...,They can't defend themselves because of their ...,en,English,0
4,86aaa48b45,ในการเล่นบทบาทสมมุติก็เช่นกัน โอกาสที่จะได้แสด...,เด็กสามารถเห็นได้ว่าชาติพันธุ์แตกต่างกันอย่างไร,th,Thai,1


In [4]:
df_test = pd.read_csv("data/test.csv")
df_test.head()

,id,premise,hypothesis,lang_abv,language
0,c6d58c3f69,بکس، کیسی، راہیل، یسعیاہ، کیلی، کیلی، اور کولم...,"کیسی کے لئے کوئی یادگار نہیں ہوگا, کولمین ہائی...",ur,Urdu
1,cefcc82292,هذا هو ما تم نصحنا به.,عندما يتم إخبارهم بما يجب عليهم فعله ، فشلت ال...,ar,Arabic
2,e98005252c,et cela est en grande partie dû au fait que le...,Les mères se droguent.,fr,French
3,58518c10ba,与城市及其他公民及社区组织代表就IMA的艺术发展进行对话&amp,IMA与其他组织合作，因为它们都依靠共享资金。,zh,Chinese
4,c32b0d16df,Она все еще была там.,"Мы думали, что она ушла, однако, она осталась.",ru,Russian


In [5]:
print("Number of training dataset : ",len(df_train))
print("Number of testing dataset  : ",len(df_test))

Number of training dataset :  12120
Number of testing dataset  :  5195


In [6]:
df_train_1 = df_train.copy()

In [17]:
y = lambda x: GoogleTranslator(source="auto",target="en").translate(x)

In [24]:
y1 = map(y,df_train["premise"][0:int(len(df_train)/25)])

In [25]:
list(y1)

['and these comments were considered in formulating the interim rules.',
 'These are issues that we wrestle with in practice groups of law firms, she said.',
 "Little things like that make a huge difference in what I'm trying to do.",
 "you know they can't really defend themselves like somebody grown uh say my age you know yeah",
 'In role play as well. Opportunities to express yourself and play multiple roles may help children capture similarities and differences between people in their desires, beliefs and feelings.',
 'You might hear someone on a farm say we have to butcher these oxen in the corral, which probably means, separate the ones to be loaded.',
 'Upon his return to the United States, Hajj was met at the airport by FBI agents, interrogated, and the next day before a federal grand jury indicted Osama bin Laden.',
 "From Cockpit Country to St. Ann's Bay",
 "Look, it's your skin, but you're going to be in trouble if you don't get busy.",
 'Every hundred degrees, the spots of p

In [ ]:
y2 = map(y,df_train["premise"][0:int(len(df_train)/25)])